# **East Asia heatwave study - analysis of model data**

In [1]:
source("../methods_extRemes.r")
source("../wwa_nonstationary_fitting.r")

# list return periods to use in later analysis
return_periods <- list("ib-ext" = 10, "ib-trop" = 10, "tl" = 10)

## **CORDEX tasmax**

In [67]:
source("../wwa_nonstationary_fitting.r")

model_results <- function(mdl, qcov_2022, qcov_hist, qcov_fut, rp = 10, lower = F, nsamp = 1000, seed = 1) {
    
    # FUNCTION TO CARRY OUT ALL ATTRIBUTION RUNS & PRODUCE RESULTS IN SPREADSHEET-FRIENDLY FORM
    
    set.seed(seed)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # FIT MODEL to three different subsets: for evaluation, attribution & projection
    
    # fit models
    df <- mdl$cov.data

    mdl_eval <- update(mdl, data = df[df$year >= 1979 & df$year <= 2022,])
    mdl_attr <- update(mdl, data = df[df$year <= 2022,])
    mdl_proj <- update(mdl, data = df[df$year <= 2050,])     
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ## EVALUATION (get model parameters)

    ci_eval <- setNames(ci.fevd(mdl_eval, type = "parameter", method = "boot")["scale",],
                        c("var_lower", "var_est", "var_upper"))

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ## ATTRIBUTION (get event value, PR & DI)

    rl_2022 <- return.level(mdl_attr, rp, qcov = qcov_2022)[1]

    pr_attr <- pextRemes(mdl_attr, rl_2022, qcov = qcov_2022, lower.tail = lower) / pextRemes(mdl_attr, rl_2022, qcov = qcov_hist, lower.tail = lower)
    dI_attr <- rl_2022 - return.level(mdl_attr, rp, qcov = qcov_hist)[1]

    boot_attr <- sapply(1:nsamp, function(i) {

        boot_df <- mdl_attr$cov.data[sample(1:length(mdl_attr$x), length(mdl_attr$x), replace = T),]
        boot_mdl <- update(mdl_attr, data = boot_df)

        boot_pr <- pextRemes(boot_mdl, rl_2022, qcov = qcov_2022, lower.tail = lower) / pextRemes(boot_mdl, rl_2022, qcov = qcov_hist, lower.tail = lower)
        boot_di <- return.level(boot_mdl, rp, qcov = qcov_2022)[1] - return.level(boot_mdl, rp, qcov = qcov_hist)[1]

        setNames(c("PR" = boot_pr, "DI" = boot_di), c("PR", "DI"))
    })
    ci_attr <- rbind("est" = c(pr_attr, dI_attr), apply(boot_attr, 1, quantile, c(0.025, 0.975), na.rm = T))
    colnames(ci_attr) <- c("PR", "DI")
    ci_attr <- unlist(lapply(colnames(ci_attr), function(cnm) setNames(ci_attr[,cnm], paste("attr", cnm, c("est", "lower", "upper"), sep = "_"))))

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ## PROJECTION (get PR & DI)

    pr_proj <- pextRemes(mdl_proj, rl_2022, qcov = qcov_2022, lower.tail = lower) / pextRemes(mdl_proj, rl_2022, qcov = qcov_fut, lower.tail = lower)
    dI_proj <- rl_2022 - return.level(mdl_proj, rp, qcov = qcov_fut)[1]

    boot_proj <- sapply(1:nsamp, function(i) {

        boot_df <- mdl_proj$cov.data[sample(1:length(mdl_proj$x), length(mdl_proj$x), replace = T),]
        boot_mdl <- update(mdl_proj, data = boot_df)

        boot_pr <- pextRemes(boot_mdl, rl_2022, qcov = qcov_2022, lower.tail = lower) / pextRemes(boot_mdl, rl_2022, qcov = qcov_fut, lower.tail = lower)
        boot_di <- return.level(boot_mdl, rp, qcov = qcov_2022)[1] - return.level(boot_mdl, rp, qcov = qcov_fut)[1]
        c("PR" = boot_pr, "DI" = boot_di)
    })
    ci_proj <- rbind("est" = c(pr_proj, dI_proj), apply(boot_proj, 1, quantile, c(0.025, 0.975), na.rm = T))
    colnames(ci_proj) <- c("PR", "DI")

    # invert
    ci_proj[,"PR"] <- 1/ci_proj[c(1,3,2),"PR"]
    ci_proj[,"DI"] <- -ci_proj[c(1,3,2),"DI"]

    ci_proj <- unlist(lapply(colnames(ci_proj), function(cnm) setNames(ci_proj[,cnm], paste("proj", cnm, c("est", "lower", "upper"), sep = "_"))))
                             
    # #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                             
    # t(setNames(data.frame(c(ci_eval, "rp_value" = actual_rl2022, ci_attr, ci_proj)), gsub(".dat", "", gsub(".+pr-mam_", "", fnm))))      
    t(data.frame(c(ci_eval, "rp_value" = rl_2022, ci_attr, ci_proj)))     
}

In [72]:
# fit model & get results for each season
nsamp <- 5
invisible(sapply(c("ib-ext", "ib-trop", "tl"), function(region) {
    
    rp <- return_periods[[region]]                  # return period depending on variable
    
    # loop over all available models, get results for each
    res <- sapply(list.files("ts", pattern = paste0("tx4x-",region), full.names = T)[1:8], function(fnm) {
        
        print(fnm)
        
        # load data
        gmst_fnm <- list.files("ts", paste0("smoothed-gsat_", strsplit(fnm, "_")[[1]][4]), full.names = T)
        df <- merge(load_ts(gmst_fnm, col.names = c("year", "gmst")), 
                    load_ts(fnm, col.names = c("year", "tx4x")))
        
        # fit model & get results
        ns_mdl <- suppressWarnings(fevd(tx4x, df[df$year >= 1979 & df$year <= 2022,], location.fun = ~gmst, type = "GEV", method = "MLE"))

        qcov_2022 <- make.qcov(ns_mdl, vals = list("mu1" = df[df$year == 2022, "gmst"]))
        qcov_hist <- make.qcov(ns_mdl, vals = list("mu1" = df[df$year == 2022, "gmst"] - 1.2))
        qcov_fut <- make.qcov(ns_mdl, vals = list("mu1" = df[df$year == 2022, "gmst"] + 0.8))
        
        data.frame(suppressWarnings(model_results(ns_mdl, qcov_2022, qcov_hist, qcov_fut, rp = rp, lower = F, nsamp = nsamp)))
        
    }, simplify = F)
    
    # merge results into single dataframe, tidy rownames & save to csv
    res <- do.call("rbind", res)
    rownames(res) <- gsub(".dat", "", gsub(paste0(".+tx4x-",region,"_"), "", rownames(res)))
    write.csv(res, paste0("res/EA-hw_tx4x-",region,"_cordex_fitted.csv"))
}))

[1] "ts/EA-hw_tx4x-ib-ext_WAS-22_EC-EARTH_r12_crCLIM-v1-1.dat"
[1] "ts/EA-hw_tx4x-ib-ext_WAS-22_HadGEM2-ES_r1_REMO2015.dat"
[1] "ts/EA-hw_tx4x-ib-ext_WAS-22_MIROC5_r1_RegCM4-7.dat"
[1] "ts/EA-hw_tx4x-ib-ext_WAS-22_MPI-ESM-LR_r1_crCLIM-v1-1.dat"
[1] "ts/EA-hw_tx4x-ib-ext_WAS-22_MPI-ESM-LR_r1_REMO2015.dat"
[1] "ts/EA-hw_tx4x-ib-ext_WAS-22_MPI-ESM-MR_r1_RegCM4-7.dat"
[1] "ts/EA-hw_tx4x-ib-ext_WAS-22_NorESM1-M_r1_crCLIM-v1-1.dat"
[1] "ts/EA-hw_tx4x-ib-ext_WAS-22_NorESM1-M_r1_RegCM4-7.dat"
[1] "ts/EA-hw_tx4x-ib-trop_WAS-22_EC-EARTH_r12_crCLIM-v1-1.dat"
[1] "ts/EA-hw_tx4x-ib-trop_WAS-22_HadGEM2-ES_r1_REMO2015.dat"
[1] "ts/EA-hw_tx4x-ib-trop_WAS-22_MIROC5_r1_RegCM4-7.dat"
[1] "ts/EA-hw_tx4x-ib-trop_WAS-22_MPI-ESM-LR_r1_crCLIM-v1-1.dat"
[1] "ts/EA-hw_tx4x-ib-trop_WAS-22_MPI-ESM-LR_r1_REMO2015.dat"
[1] "ts/EA-hw_tx4x-ib-trop_WAS-22_MPI-ESM-MR_r1_RegCM4-7.dat"
[1] "ts/EA-hw_tx4x-ib-trop_WAS-22_NorESM1-M_r1_crCLIM-v1-1.dat"
[1] "ts/EA-hw_tx4x-ib-trop_WAS-22_NorESM1-M_r1_RegCM4-7.dat"
[1] "ts/EA-h